In [1]:
import cv2
import torch
import numpy as np
import math
from numpy import random
import torchvision.models as torch_models
import os
from tqdm.auto import tqdm
# from deep_sort_pytorch.utils.parser import get_config
# from deep_sort_pytorch.deep_sort import DeepSort
from torchvision import transforms
from PIL import Image
from torch import nn
import logging
from utils import *
from super_gradients.training import models
from base64 import b64encode

logging.getLogger("super_gradients").setLevel(logging.WARNING)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps") if torch.backends.mps.is_available() else device
print(device)
!nvidia-smi

/mnt/netapp_data/bighomes-active-netapp/zw2688/environments/vision/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The console stream is logged into /homedir/ugrad/z/zw2688/sg_logs/console.log


[2023-11-15 14:56:41] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2023-11-15 14:56:41] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-11-15 14:56:44] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-11-15 14:56:44] WARNING - export.py - Failed to import pytorch_quantization
[2023-11-15 14:56:44] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization


cuda
Wed Nov 15 14:56:44 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-16GB           Off | 00000000:13:00.0 Off |                    0 |
| N/A   35C    P0              38W / 250W |    742MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+--------------------

In [2]:
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])


cls_model_chkpoint_path = "cls_chkpoint/checkpoint_2023-11-09-12-07/checkpoint_epoch_5.pth"

cls_model = torch_models.resnet18(weights=torch_models.ResNet18_Weights.DEFAULT)    
load_checkpoint(cls_model, checkpoint_path = "cls_chkpoint/checkpoint_2023-11-09-12-07/best_model.pth", )

cls_model.to(device)
cls_model.eval();


In [3]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
# video_path = './clips/20230720_Game4.mp4'
# video_path = 'videos/2023_11_15_id_jM9CfUbroAg.mp4'
video_path = 'clips/clip_2.mp4'
cap, fps, frame_width, frame_height = get_video_info(video_path)
classNames = ['basketball', 'basketball-hoops', 'made']
cap = cv2.VideoCapture(video_path)

out=cv2.VideoWriter('output.mp4', fourcc, fps, (frame_width, frame_height))
score = 0
model = models.get("yolo_nas_s", num_classes=3, checkpoint_path="./yolonas_checkpoints/bball_detect/RUN_20231104_130305_141167/ckpt_best.pth").to(device)
i = 0
last_score_update_time = -1
start_time = 0
end_time = np.inf
cap.set(cv2.CAP_PROP_POS_MSEC, start_time)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
pbar = tqdm(total=total_frames, desc="Processing Frames", unit="frame")
model.prep_model_for_conversion(input_size=(640, 640))
while True:
    ret, frame = cap.read()
    #frame = cv2.resize(frame, (2 * frame_width // 3, 2 * frame_height // 3) )
    frame_start_time = time.time()
    current_time = cap.get(cv2.CAP_PROP_POS_MSEC)
    if ret and current_time < end_time:
        result=list(model.predict(frame, conf=0.5))[0]
        bbox_xyxys=result.prediction.bboxes_xyxy.tolist()
        confidences=result.prediction.confidence
        labels=result.prediction.labels.tolist()
        frame_copy = frame.copy()
        for (bbox_xyxy, confidence, cls) in zip(bbox_xyxys, confidences, labels):
            bbox=np.array(bbox_xyxy)
            x1, y1, x2, y2 = bbox[0], bbox[1], bbox[2], bbox[3]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            classname=int(cls)
            class_name=classNames[classname]
            center_x_bbox = (x1 + x2) / 2
            if class_name == "basketball-hoops":
                
                if x1 > x2 or y1 > y2:
                    continue
                cropped_img, predicted = predict_hoop_box(frame_copy, cls_model, x1, y1, x2, y2, preprocess = preprocess, device = device)
                if predicted == 1:
                    cropped_img.save(f"cropped/score_{score}.jpg")
                if predicted == 1 and current_time - last_score_update_time > 1000:
                    score += 1
                    last_score_update_time = current_time
                    
            conf=round(confidence, 2)
            label=f'{class_name}{conf}'

            cv2.rectangle(frame, (x1, y1), (x2, y2), color=(0, 0, 255, 0.6),thickness=2, lineType=cv2.LINE_AA)
            #Create a rectangle above the detected object and add label and confidence score
            t_size=cv2.getTextSize(str(label), 0, fontScale=1/2, thickness=1)[0]
            c2=x1+t_size[0], y1-t_size[1]-3
            cv2.rectangle(frame, (x1, y1), c2, color=(0, 0, 255, 0.6), thickness=-1, lineType=cv2.LINE_AA)
            cv2.putText(frame, str(label), (x1, y1-2), 0, 1/2, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)
            cv2.putText(frame, f'Score: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        #cv2.imshow("Frame", frame)
        out.write(frame)
        frame_end_time = time.time()  # End time for frame processing
        time_per_frame = frame_end_time - frame_start_time
        pbar.set_postfix(time_per_frame=f"{time_per_frame:.3f} sec")
        pbar.update(1)
        #print(f'\rProcessing frame: {i + 1}/{total_frames}\t{time_per_frame:.3f} s/frame', end='')
        i += 1
    else:
        break

out.release()
cap.release()

Processing Frames:   0%|          | 0/199 [00:00<?, ?frame/s]/mnt/netapp_data/bighomes-active-netapp/zw2688/environments/vision/lib/python3.9/site-packages/numpy/lib/arraypad.py:487: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.array(x)
Processing Frames:  12%|█▏        | 23/199 [00:04<00:21,  8.23frame/s, time_per_frame=0.117 sec]

KeyboardInterrupt: 

In [15]:
display_compressed_video('output.mp4')

FileNotFoundError: [Errno 2] No such file or directory: 'result_compressed.mp4'

sh: ffmpeg: command not found


In [5]:
display_video("videos/2023_11_15_id_jM9CfUbroAg.mp4")

In [30]:
model.eval()
example = torch.rand(1, 3, 224, 224).to(device)
traced_script_module = torch.jit.trace(model, example)
traced_script_module.save("yolonas.pt")

In [6]:
def initialize_deepsort():
    # Create the Deep SORT configuration object and load settings from the YAML file
    cfg_deep = get_config()
    cfg_deep.merge_from_file("deep_sort_pytorch/configs/deep_sort.yaml")

    # Initialize the DeepSort tracker
    deepsort = DeepSort(cfg_deep.DEEPSORT.REID_CKPT,
                        max_dist=cfg_deep.DEEPSORT.MAX_DIST,
                        # min_confidence  parameter sets the minimum tracking confidence required for an object detection to be considered in the tracking process
                        min_confidence=cfg_deep.DEEPSORT.MIN_CONFIDENCE,
                        #nms_max_overlap specifies the maximum allowed overlap between bounding boxes during non-maximum suppression (NMS)
                        nms_max_overlap=cfg_deep.DEEPSORT.NMS_MAX_OVERLAP,
                        #max_iou_distance parameter defines the maximum intersection-over-union (IoU) distance between object detections
                        max_iou_distance=cfg_deep.DEEPSORT.MAX_IOU_DISTANCE,
                        # Max_age: If an object's tracking ID is lost (i.e., the object is no longer detected), this parameter determines how many frames the tracker should wait before assigning a new id
                        max_age=cfg_deep.DEEPSORT.MAX_AGE, n_init=cfg_deep.DEEPSORT.N_INIT,
                        #nn_budget: It sets the budget for the nearest-neighbor search.
                        nn_budget=cfg_deep.DEEPSORT.NN_BUDGET,
                        use_cuda=True
        )

    return deepsort

deepsort = initialize_deepsort()

[2023-11-02 15:03:02] INFO - feature_extractor.py - Loading weights from deep_sort_pytorch/deep_sort/deep/checkpoint/ckpt.t7... Done!


In [7]:
names = classNames
colors = [[random.randint(0, 255) for _ in range(3)]
          for _ in range(len(names))]
palette = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)

In [19]:
def compute_color_for_labels(class_name):
    """
    Function that adds fixed color depending on the class
    """
    if class_name == 'person':
        color = (0, 0, 255)  # Red color for 'person'
    elif class_name == 'sports ball':
        color = (0, 255, 0)  # Green color for 'sports ball'
    else:
        color = (0, 0, 255)  
        
    return tuple(color) 

def draw_boxes(frame, bbox, identities=None, categories=None, names=None, offset=(0,0)):
    for i, box in enumerate(bbox):
        x1, y1,  x2, y2 = [int(i) for i in box]
        x1 += offset[0]
        x2 += offset[0]
        y1 += offset[0]
        y2 += offset[0]
        cat = int(categories[i]) if categories is not None else 0
        cat = classNames[cat]
        id = int(identities[i]) if identities is not None else 0
        #Create Bounding Boxes around the Detected Objects
        cv2.rectangle(frame, (x1, y1), (x2, y2), color= compute_color_for_labels(cat),thickness=2, lineType=cv2.LINE_AA)
        label = str(id) + ":" + cat
        (w,h), _ = cv2.getTextSize(str(label), cv2.FONT_HERSHEY_SIMPLEX, fontScale=1/2, thickness=1)
        #Create a rectangle above the detected object and add label and confidence score
        t_size=cv2.getTextSize(str(label), cv2.FONT_HERSHEY_SIMPLEX, fontScale=1/2, thickness=1)[0]
        c2=x1+t_size[0], y1-t_size[1]-3
        cv2.rectangle(frame, (x1, y1), c2, color=compute_color_for_labels(cat), thickness=-1, lineType=cv2.LINE_AA)
        cv2.putText(frame, str(label), (x1, y1-2), 0, 1/2, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)
    return frame

In [20]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
output=cv2.VideoWriter('tracking_out.mp4', fourcc, fps*0.6, (frame_width, frame_height))
while True:
    xywh_bboxs = []
    confs = []
    oids = []
    outputs = []
    ret, frame = cap.read()
    if ret:
        result = list(model.predict(frame, conf=0.5))[0]
        bbox_xyxys = result.prediction.bboxes_xyxy.tolist()
        confidences = result.prediction.confidence
        labels = result.prediction.labels.tolist()
        for (bbox_xyxy, confidence, cls) in zip(bbox_xyxys, confidences, labels):
            bbox = np.array(bbox_xyxy)
            x1, y1, x2, y2 = bbox[0], bbox[1], bbox[2], bbox[3]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            conf = math.ceil((confidence*100))/100
            cx, cy = int((x1+x2)/2), int((y1+y2)/2)
            bbox_width = abs(x1-x2)
            bbox_height = abs(y1-y2)
            xcycwh = [cx, cy, bbox_width, bbox_height]
            xywh_bboxs.append(xcycwh)
            confs.append(conf)
            oids.append(int(cls))
        xywhs = torch.tensor(xywh_bboxs)
        confss= torch.tensor(confs)
        outputs = deepsort.update(xywhs, confss, oids, frame)
        if len(outputs)>0:
            bbox_xyxy = outputs[:,:4]
            identities = outputs[:, -2]
            object_id = outputs[:, -1]
            draw_boxes(frame, bbox_xyxy, identities, object_id)
        output.write(frame)
    else:
        break

output.release()
cap.release()

In [4]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
output=cv2.VideoWriter('tracking_out.mp4', fourcc, fps*0.6, (frame_width, frame_height))
while True:
    xywh_bboxs = []
    confs = []
    oids = []
    outputs = []
    ret, frame = cap.read()
    if ret:
        result = list(model(frame, 
                        stream = True, 
                        device = device, 
                        conf = 0.4))[0]
        bbox_xyxys = result.prediction.bboxes_xyxy.tolist()
        confidences = result.prediction.confidence
        labels = result.prediction.labels.tolist()
        for (bbox_xyxy, confidence, cls) in zip(bbox_xyxys, confidences, labels):
            bbox = np.array(bbox_xyxy)
            x1, y1, x2, y2 = bbox[0], bbox[1], bbox[2], bbox[3]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            conf = math.ceil((confidence*100))/100
            cx, cy = int((x1+x2)/2), int((y1+y2)/2)
            bbox_width = abs(x1-x2)
            bbox_height = abs(y1-y2)
            xcycwh = [cx, cy, bbox_width, bbox_height]
            xywh_bboxs.append(xcycwh)
            confs.append(conf)
            oids.append(int(cls))
        xywhs = torch.tensor(xywh_bboxs)
        confss= torch.tensor(confs)
        outputs = deepsort.update(xywhs, confss, oids, frame)
        if len(outputs)>0:
            bbox_xyxy = outputs[:,:4]
            identities = outputs[:, -2]
            object_id = outputs[:, -1]
            draw_boxes(frame, bbox_xyxy, identities, object_id)
        output.write(frame)
    else:
        break

output.release()
cap.release()

NameError: name 'fps' is not defined

In [31]:
display_video("output.mp4")

In [8]:
model = models.get("yolo_nas_s", pretrained_weights='coco')
model.load_state_dict(torch.load("yolonas_checkpoints/bball_detect/RUN_20231104_130305_141167/ckpt_best.pth"))

Downloading: "https://sghub.deci.ai/models/yolo_nas_s_coco.pth" to /homedir/ugrad/z/zw2688/.cache/torch/hub/checkpoints/yolo_nas_s_coco.pth
100%|██████████| 73.1M/73.1M [00:02<00:00, 25.9MB/s]


RuntimeError: Error(s) in loading state_dict for YoloNAS_S:
	Missing key(s) in state_dict: "backbone.stem.conv.branch_3x3.conv.weight", "backbone.stem.conv.branch_3x3.bn.weight", "backbone.stem.conv.branch_3x3.bn.bias", "backbone.stem.conv.branch_3x3.bn.running_mean", "backbone.stem.conv.branch_3x3.bn.running_var", "backbone.stem.conv.branch_1x1.weight", "backbone.stem.conv.branch_1x1.bias", "backbone.stem.conv.post_bn.weight", "backbone.stem.conv.post_bn.bias", "backbone.stem.conv.post_bn.running_mean", "backbone.stem.conv.post_bn.running_var", "backbone.stem.conv.rbr_reparam.weight", "backbone.stem.conv.rbr_reparam.bias", "backbone.stage1.downsample.branch_3x3.conv.weight", "backbone.stage1.downsample.branch_3x3.bn.weight", "backbone.stage1.downsample.branch_3x3.bn.bias", "backbone.stage1.downsample.branch_3x3.bn.running_mean", "backbone.stage1.downsample.branch_3x3.bn.running_var", "backbone.stage1.downsample.branch_1x1.weight", "backbone.stage1.downsample.branch_1x1.bias", "backbon